In [1]:
%load_ext autoreload
%autoreload 2

from module_test.raw_code.optionlib_2.pricing.binomial import (
    VectorBinomialCRR,
    NodeBinomialCRR,
    EquityForward,
    MarketBinomial,
    crr_binomial_pricing,
    NodeBinomialLR,
    VectorBinomialLR,
    time_distance_helper
)
from trade.helpers.helper import change_to_last_busday, retrieve_timeseries
from module_test.raw_code.optionlib_2.assets.dividend import Schedule
def get_spot(tick, date):
    return retrieve_timeseries(tick, date, date)['close'][0]


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-07-25 00:26:27 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://54.144.4.219:5500/thetadata


In [4]:
Schedule([])

<Schedule: 0 dividends>

In [17]:
# 2025-07-16	AAPL	2026-12-18	450.0	C	8	0.44	18	0.48	20250716	0.460	0.467692	0.269251
test_start, test_valuation_date = '2025-07-16', '2025-07-16'
start, valuation_date = test_start, test_valuation_date
strike = 450.0
opt_type = 'c'
mid = 0.460
vol = 0.269251
exp_date = '2026-12-18'
S = get_spot('AAPL', (test_valuation_date))

mkt_forward = EquityForward(
    start_date=start,
    end_date=exp_date,
    ticker='AAPL',
    valuation_date=valuation_date,
    risk_free_rate=None,
    dividend_type='discrete',
    dividend=None,  # Market dividend will be set later
)

mkt_forward_cont = EquityForward(
    start_date=start,
    end_date=exp_date,
    ticker='AAPL',
    valuation_date=valuation_date,
    risk_free_rate=None,
    dividend_type='continuous',
    dividend=None,  # Market dividend will be set later

)
# get_spot('AAPL', start)
rates = mkt_forward.risk_free_rate
mkt_forward_cont.dividend.yield_rate

0.0036162922856715937

In [20]:
mkt_bt = MarketBinomial(
    tick='AAPL',
    K=strike,
    expiration=exp_date,
    sigma=vol,
    N=500,
    dividend_type='discrete',
    option_type=opt_type,
    start_date=start,
    valuation_date=valuation_date,
    american=True
)

mkt_bt.price()

0.474859103593949

In [23]:
(mkt_bt.forward.dividend.schedule)

In [15]:
mkt_bt.delta()

0.02056167769032562

In [4]:
mkt_bt.asset.spot_price

210.16000366210938

In [5]:
crr_binomial_pricing(
    K=strike,
    sigma=vol,
    r=mkt_forward.risk_free_rate,
    N=500,
    spot_price=mkt_bt.asset.spot_price,
    dividend_type='continuous',
    option_type=opt_type,
    T= time_distance_helper(
        mkt_forward.end_date, mkt_forward.valuation_date
    ),
    american=True,
    div_amount=mkt_forward_cont.dividend.yield_rate
)

0.46254616656251185

In [10]:
bt = NodeBinomialLR(
    K=strike,
    expiration=exp_date,
    sigma=vol,
    r=rates,
    N=200,
    spot_price=S,
    dividend_type='discrete',
    div_amount=mkt_forward.dividend.get_year_fractions(),
    option_type=opt_type,
    start_date=start,
    valuation_date=valuation_date,
    american=True
)
# bt.price()
bt.tree
bt.price(),bt.d, bt.u, bt.p

(7.679119748243995e-06,
 0.9775391951331914,
 1.0229768841787958,
 0.3999360964036095)

In [11]:
bt = VectorBinomialLR(
    K=strike,
    expiration=exp_date,
    sigma=vol,
    r=rates,
    N=200,
    spot_price=S,
    dividend_type='discrete',
    div_amount=mkt_forward.dividend.get_year_fractions(),
    option_type='c',
    start_date=start,
    valuation_date=valuation_date,
    american=True
)
# bt.price()
bt.price(),bt.d, bt.u, bt.p

# bt.price()


(7.679119748243995e-06,
 0.9775391951331914,
 1.0229768841787958,
 0.3999360964036095)

In [12]:
bt = VectorBinomialCRR(
    K=strike,
    expiration=exp_date,
    sigma=vol,
    r=rates,
    N=1000,
    spot_price=S,
    dividend_type='discrete',
    div_amount=mkt_forward.dividend.get_year_fractions(),
    option_type=opt_type,
    start_date=start,
    valuation_date=valuation_date,
    american=True
)
# bt.price()
bt.tree
bt.price(),bt.d, bt.u, bt.p


(0.4763023112933306,
 0.9898921290700986,
 1.0102110832413596,
 0.5004255089632036)

In [13]:
bt = NodeBinomialCRR(
    K=strike,
    expiration=exp_date,
    sigma=vol,
    r=rates,
    N=1000,
    spot_price=S,
    dividend_type='discrete',
    div_amount=mkt_forward.dividend.get_year_fractions(),
    option_type=opt_type,
    start_date=start,
    valuation_date=valuation_date,
    american=True
)
# bt.price()
# bt.tree
# bt.price()
bt.price(),bt.d, bt.p, bt.u

(0.47630231129334016,
 0.9898921290700985,
 0.5004255089632063,
 1.0102110832413596)

In [14]:
bt.delta(), bt.gamma(), bt.vega(), bt.theta(), bt.rho()

(Scalar(0.021141333457329464),
 Scalar(0.0007514823731175563),
 0.1290868060044704,
 -0.0,
 0.056421028182442684)